# Cleaning & Preprocessing

## Imports

In [1]:
# Import dependencies
import pandas as pd
import uuid
from sqlalchemy import create_engine

## Cleaning

In [2]:
# Read CSV
filepath = 'resources/mxmh_survey_results.csv'
df = pd.read_csv(filepath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Timestamp                     736 non-null    object 
 1   Age                           735 non-null    float64
 2   Primary streaming service     735 non-null    object 
 3   Hours per day                 736 non-null    float64
 4   While working                 733 non-null    object 
 5   Instrumentalist               732 non-null    object 
 6   Composer                      735 non-null    object 
 7   Fav genre                     736 non-null    object 
 8   Exploratory                   736 non-null    object 
 9   Foreign languages             732 non-null    object 
 10  BPM                           629 non-null    float64
 11  Frequency [Classical]         736 non-null    object 
 12  Frequency [Country]           736 non-null    object 
 13  Frequ

In [3]:
# Function to reformat columns lowercase w/o spaces/braces
def col_mapper(old):
    lower = old.lower()
    underscore = lower.replace(' ', '_')
    new = underscore.replace('[', '').replace(']', '') # 1 level of abstraction
    return new

In [4]:
df.rename(columns=col_mapper, inplace=True)
print(df.columns)

Index(['timestamp', 'age', 'primary_streaming_service', 'hours_per_day',
       'while_working', 'instrumentalist', 'composer', 'fav_genre',
       'exploratory', 'foreign_languages', 'bpm', 'frequency_classical',
       'frequency_country', 'frequency_edm', 'frequency_folk',
       'frequency_gospel', 'frequency_hip_hop', 'frequency_jazz',
       'frequency_k_pop', 'frequency_latin', 'frequency_lofi',
       'frequency_metal', 'frequency_pop', 'frequency_r&b', 'frequency_rap',
       'frequency_rock', 'frequency_video_game_music', 'anxiety', 'depression',
       'insomnia', 'ocd', 'music_effects', 'permissions'],
      dtype='object')


In [5]:
# Add unique id column
df['uuid'] = [uuid.uuid4() for _ in range(len(df))]

# move it to first
first_col = df.pop('uuid')
df.insert(0, 'uuid', first_col)

df.head(2)

,uuid,timestamp,age,primary_streaming_service,hours_per_day,while_working,instrumentalist,composer,fav_genre,exploratory,...,frequency_r&b,frequency_rap,frequency_rock,frequency_video_game_music,anxiety,depression,insomnia,ocd,music_effects,permissions
0,c5f3024e-e31a-4a83-89ac-f83d8be5b588,8/27/2022 19:29:02,18.0,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,...,Sometimes,Very frequently,Never,Sometimes,3.0,0.0,1.0,0.0,NaN,I understand.
1,fea9f95b-cb86-4b37-b4f3-478d0ccdaa2c,8/27/2022 19:57:31,63.0,Pandora,1.5,Yes,No,No,Rock,Yes,...,Sometimes,Rarely,Very frequently,Rarely,7.0,2.0,2.0,1.0,NaN,I understand.


In [6]:
# Replace improve with 1
df = df.replace({
    'Improve': 1,
    'No effect': 0,
    'Worsen': 0 # too few responses
})
df['music_effects'][0:5]

0    NaN
1    NaN
2    0.0
3    1.0
4    1.0
Name: music_effects, dtype: float64

In [7]:
# Remove unhelpful and potentially PII columns
cols_to_drop = [
    'bpm', # too few responses and too nonsensical answers
    'permissions', # everyone answered yes
    'timestamp' # potentially PII
    ]

cleaned_df = df.drop(columns=cols_to_drop)

In [8]:
# Drop remaining rows containing any number of NaN
cleaned_df = cleaned_df.dropna()
print(f'{len(df)-len(cleaned_df)} rows dropped ({len(cleaned_df)} remaining)')

18 rows dropped (718 remaining)


## Export to SQL

In [9]:
cleaned_df['uuid'] = cleaned_df['uuid'].astype(str)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 718 entries, 2 to 735
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uuid                        718 non-null    object 
 1   age                         718 non-null    float64
 2   primary_streaming_service   718 non-null    object 
 3   hours_per_day               718 non-null    float64
 4   while_working               718 non-null    object 
 5   instrumentalist             718 non-null    object 
 6   composer                    718 non-null    object 
 7   fav_genre                   718 non-null    object 
 8   exploratory                 718 non-null    object 
 9   foreign_languages           718 non-null    object 
 10  frequency_classical         718 non-null    object 
 11  frequency_country           718 non-null    object 
 12  frequency_edm               718 non-null    object 
 13  frequency_folk              718 non-null

In [10]:
# Initialize SQLAlchemy engine
engine = create_engine('sqlite:///resources/cleaned.db')

In [11]:
cleaned_df.to_sql('main', con=engine, if_exists='replace', index=False)

718